## Import required packages

In [1]:
from typing import Dict, List, Tuple

import tqdm

import numpy as np
import pandas as pd

import scipy
import scipy.stats

import seaborn as sns
import matplotlib.pyplot as plt

import pathlib
import tttrlib

import os

from feda_tools import twodim_hist as tdh
from feda_tools import utilities as utils
from feda_tools import analysis as an

from decimal import Decimal, getcontext

import numpy.ma as ma
from scipy.stats import norm
from scipy.stats import halfnorm

## Load target PTU file

In [2]:
# Load PTU Files
# file_path = pathlib.Path('//130.127.188.19/projects/FoxP_FKH-DNA/20220314_FoxP1_data_all_NK/20220310_V78C_monomer_NK/V78C_monomer_FoxP1_1hr/burstwise_All 0.2027#30')
# bid_path = pathlib.Path('//130.127.188.19/projects/FoxP_FKH-DNA/20220314_FoxP1_data_all_NK/20220310_V78C_monomer_NK/V78C_monomer_FoxP1_1hr/burstwise_All 0.2027#30/BIDs_30ph')

### for testing purposes ###
# file_path = pathlib.Path('C:/Users/2administrator/Documents/source/repos/feda_tools/test data/2022/03_02_22_Troubleshooting_detection_efficiencies/Combined_old_thresholds/Split_After_Adjust_HF_54000s_pinhole6-000000.ptu')

#total time 816.9 seconds for this file
file_p ='C:/Users/2administrator/Documents/source/repos/feda_tools/test data/2022/03_02_22_Troubleshooting_detection_efficiencies/Combined_old_thresholds/Split_After_Adjust_HF_54000s_pinhole6-000000.ptu'

## Initialize tttrlib data and extract important global data

In [50]:
data = tttrlib.TTTR(file_p, 'PTU')
all_macro_times = data.macro_times
all_micro_times = data.micro_times
routing_channels =  data.routing_channels

#in seconds. usually the first plots are in ms to see the bursts.
macro_res =data.get_header().macro_time_resolution
micro_res = data.get_header().micro_time_resolution

## Calculate photon time intervals

In [4]:
#iterate through macro and micro times to calculate delta time between photon events
arr_size = len(all_macro_times) - 1
photon_time_intervals = np.zeros(arr_size, dtype = np.float64)
lw = 0.25
for i in range(0, len(photon_time_intervals)):
    photon_1 = (all_macro_times[i]*macro_res) + (all_micro_times[i]*micro_res)
    photon_2 = (all_macro_times[i+1]*macro_res) + (all_micro_times[i+1]*micro_res)
    photon_time_intervals[i] = (photon_2 - photon_1)*1000

# create photon ID array
photon_ids = np.arange(1, arr_size + 1)


In [5]:
print(photon_time_intervals[-1])

2.3073616332567326


## Plot the Raw Data

In [39]:
# plot interactive
%matplotlib qt

# raw data, no running average
plt.plot(photon_ids, np.log(photon_time_intervals), linewidth = 0.25)
plt.xlim(0, 91000)
plt.show()

In [37]:
# Calculate the running average
cumulative_sum = np.cumsum(photon_time_intervals)
running_average = cumulative_sum / np.arange(1, len(photon_time_intervals) + 1)

In [8]:
plt.plot(photon_ids, running_average)

## Plot the log of the running average

In [42]:
def running_average(data, window_size):
    window = np.ones(window_size) / window_size
    return np.convolve(data, window, mode='valid')

# Create a NumPy array
data = photon_time_intervals

# Set the window size for the running average
window_size = 30

# Calculate the running average
running_avg = running_average(data, window_size)



In [43]:
# Plot the running average
# plt.plot(data, label='Original Data')
lw = 0.25
xarr = np.arange(window_size - 1, len(data))
logrunavg = np.log(running_avg)
plt.plot(xarr, logrunavg, label='Running Average', linewidth = lw)
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.xlim(0, 91000)
plt.show()

## Plot the log of the running average using 2D heat map to better visualize noise

In [45]:
# plot the running average as a 2D histogram with 1D histograms on the margins

bins = {"x":141, "y": 141}
xrange = {"min" : 0, "max" : 91000}
yrange = {"min" : -6, "max" : 2}
fig, ax, twodimdata = tdh.make_plot(xarr, logrunavg, "x", "y",xrange ,yrange, bins)

## Visually estimate the norm center and visualize the best fit

In [31]:
### estimate the half gaussian fit on the top half of the data.
# set the threshold based on visual inspection. here, threshold ~ mu
threshold_value = -.34

# compress the filtered data to remove the masked values
filtered_logrunavg = ma.masked_less(logrunavg, threshold_value).compressed()

# Set all masked values to zero
counts_logrunavg, bins_logrunavg, _ = plt.hist(logrunavg, bins = bins['y'], alpha=0.6, color='r')
plt.hist(filtered_logrunavg, bins = bins_logrunavg, alpha=0.6, color='b')
plt.show()

In [32]:
# fit with halfnorm. visualize for best fit testing. get mu and std dev
mu, std = halfnorm.fit(filtered_logrunavg)

# counts_logrunavg, bins_logrunavg, _ = plt.hist(logrunavg, bins = bins['y'], density= True, alpha=0.6, color='r')
plt.hist(filtered_logrunavg, bins = bins['y'], density = True, alpha=0.6, color='r')

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = halfnorm.pdf(x, mu, std)

plt.plot(x, p, 'k', linewidth=2)
title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
plt.title(title)

# Display the plot
plt.show()

## Using the halfnorm fit, define 2sigma threshold and plot for inspection

In [46]:
### Using std from halfnorm fit, set the threshold for filtering out noise. Then, filter out noise.
threshold_value = mu - 2*std
filtered_values = ma.masked_greater(logrunavg, threshold_value)

In [47]:
# visualize the log running average and the threshold values
plt.plot(xarr, logrunavg, label='Running Average', linestyle='None', marker = 'o', markersize = 5)
plt.plot(xarr, filtered_values, label='Threshold Values', linestyle='None', marker = '.', markersize = 5)
plt.xlabel('Photon Event #')
plt.ylabel('log(Photon Interval Time)')
plt.legend()
plt.xlim(0, 91000)
plt.show()

## Extract the photon data that meets the thresdhold condition and create a burst index.

In [16]:
### define function for extracting the unmasked segments from the thresholded data.
def extract_unmasked_indices(masked_array):
    unmasked_indices_lists = []
    current_indices = []

    # iterate through masked array and collect unmasked index segments
    for i, value in enumerate(masked_array):
        if np.ma.is_masked(value):
            if current_indices:
                unmasked_indices_lists.append(current_indices)
                current_indices = []
        else:
            current_indices.append(i)

    # handle the last segment
    if current_indices:
        unmasked_indices_lists.append(current_indices)

    return unmasked_indices_lists

In [17]:
### Get a burst index. Each list is a burst, and each list contains the indices of 
### the photon events in the original data.
burst_index = extract_unmasked_indices(filtered_values)

## Create bi4_bur dataframe and calculate statistics

In [52]:
################################
### create bi4_bur dataframe ###
################################
min_photon_count = 100

# prepare empty dataframe to calculate values
bi4_bur_df = pd.DataFrame(columns=
                          ['First Photon', 
                           'Last Photon', 
                           'Duration (ms)', 
                           'Mean Macro Time (ms)', 
                           'Number of Photons', 
                           'Count Rate (kHz)'])

### calculate each burst record and store in df
for burst in burst_index:

    # filter out bursts with one or less photons.
    if len(burst) <= min_photon_count:
        continue

    #############################
    ### First and Last Photon ###
    #############################
    
    first_photon = burst[0]
    last_photon = burst[-1]

    ##########################
    ### Duration (ms) calc ###
    ##########################
    
    lp_time = all_macro_times[last_photon]*macro_res + all_micro_times[last_photon]*micro_res
    fp_time = all_macro_times[first_photon]*macro_res + all_micro_times[first_photon]*micro_res
    lp_time_ms = lp_time*1000
    fp_time_ms = fp_time*1000
    duration = (lp_time_ms - fp_time_ms)

    #################################
    ### Mean Macro Time (ms) Calc ###
    #################################
    
    # get all the macro times corresponding to the photons in this burst
    macro_times = all_macro_times[burst[0]]*macro_res*1000
    
    # calculate the mean
    mean_macro_time = np.mean(macro_times)

    ##############################
    ### Number of Photons calc ###
    ##############################
    num_photons = len(burst)

    #######################
    ### Count Rate calc ###
    #######################
    count_rate = num_photons / duration

    #################################
    ### Duration (green)(ms) calc ###
    #################################
    
    # Assuming you have your list of indexes
    list_of_indexes = burst
    
    # Get the routing channel per index data
    routing_channels = data.routing_channels
    
    # Create boolean masks for channels 0 and 2 to select the green channels
    mask_channel_0 = routing_channels[list_of_indexes] == 0
    mask_channel_2 = routing_channels[list_of_indexes] == 2
    
    # Use boolean masks to filter the indexes
    indexes_channel_0 = np.array(list_of_indexes)[mask_channel_0]
    indexes_channel_2 = np.array(list_of_indexes)[mask_channel_2]
    
    # Output the filtered indexes
    # print("Indexes corresponding to channel 0:", indexes_channel_0)
    # print("Indexes corresponding to channel 2:", indexes_channel_2)
    
    # Find the minimum and maximum indexes across both resulting arrays. These are first and last green 
    # photon in this burst. handle situations where only one channel has photons or none have photons.
    if len(indexes_channel_0) > 0 and len(indexes_channel_2) > 0:
        first_green_photon = min(np.min(indexes_channel_0), np.min(indexes_channel_2))
        last_green_photon = max(np.max(indexes_channel_0), np.max(indexes_channel_2))
    elif len(indexes_channel_0) >= 2:
        first_green_photon = np.min(indexes_channel_0)
        last_green_photon = np.max(indexes_channel_0)
    elif len(indexes_channel_2) >= 2:
        first_green_photon = np.min(indexes_channel_2)
        last_green_photon = np.max(indexes_channel_2)
    else:
        first_green_photon = None
        last_green_photon = None

    # Calculate duration or set as NaN
    if first_green_photon != None and last_green_photon != None:
        lgp_time = all_macro_times[last_green_photon]*macro_res + all_micro_times[last_green_photon]*micro_res
        fgp_time = all_macro_times[first_green_photon]*macro_res + all_micro_times[first_green_photon]*micro_res
        lgp_time_ms = lgp_time*1000
        fgp_time_ms = fgp_time*1000
        duration_green = (lgp_time_ms - fgp_time_ms)
    else:
        duration_green = np.nan

    #########################################
    #### Mean Macro Time (green)(ms) calc ###
    #########################################
    
    # get all the macro times corresponding to the photons in this burst
    macro_times_ch0 = all_macro_times[indexes_channel_0]*macro_res*1000
    macro_times_ch2 = all_macro_times[indexes_channel_2]*macro_res*1000
    
    # Concatenate the arrays along the appropriate axis
    combined_macro_times = np.concatenate([macro_times_ch0, macro_times_ch2], axis=0)
    
    # calculate the mean
    mean_macro_time_green = np.mean(combined_macro_times, axis=0)

    #################################
    ### Number of Photons (green) ###
    #################################
    num_photons_gr = len(indexes_channel_0) + len(indexes_channel_2)

    ########################
    ### Green Count Rate ###
    ########################
    count_rate_gr = num_photons_gr / duration_green
    
    new_row = {'First Photon': [first_photon],'Last Photon': [last_photon],
               'Duration (ms)': [duration],'Mean Macro Time (ms)': [mean_macro_time],
               'Number of Photons': [num_photons], 'Count Rate (kHz)': [count_rate],
               'Duration (green) (ms)': [duration_green], 
               'Mean Macro Time (green) (ms)': [mean_macro_time_green],
               'Number of Photons (green)': [num_photons_gr],
               'Green Count Rate (kHz)': count_rate_gr
              }
    
    new_record = pd.DataFrame.from_dict(new_row)
    
    ### append record to df
    bi4_bur_df = pd.concat([bi4_bur_df, new_record], ignore_index=True)
    

C:\Users\2administrator\AppData\Local\Temp\ipykernel_27264\2822540145.py:142: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bi4_bur_df = pd.concat([bi4_bur_df, new_record], ignore_index=True)
